###### **Machine Learning book on Jupyter NBs**
## Recurrent Neural Networks & LSTM

**Contents:**

* RNN an intuitive description
* RNN a more formal description
* LSTM: Long Short Term Memory Networks 


### 1. An intuitive description

Recurrent ANN or RNN are expecially tailored to work with sequential data (e.g. time series, text mining, speech processing, NLP, ecc.). In practice this means that they can be used to deal with 1-D data, i.e. ordered sequences where the ordering is usally defined by the temporal variable (but non exclusively). They however can also be used RNN to deal with 2D data, where you have two directions.

The topic of RNN is strictly related to that of the Hopfield Machines [ArRNN.1]. RNN were first introduced in  1988 in the paper [ArRNN.2] and they are commonly attributed to David Everett Rumelhart (1942 – 2011). He was an American psychologist who made many contributions to the formal analysis of human cognition, working primarily within the frameworks of mathematical psychology, symbolic artificial intelligence, and parallel distributed processing. He also admired formal linguistic approaches to cognition, and explored the possibility of formulating a formal grammar to capture the structure of stories. Unfortunately he died quite young by a neurodegenerative disease. 

<img src="immagini_ANN/Rumelhart.jpg" alt="drawing" width="20%"/>

The extremely good performances of RNN on series have lead to numerous evolutions such as LSTM and Transformers, which have revolutionised the fields of speech processing, text mining, genomics, ecc. We shall quickly explore the main aspects of all these architectures. 

For an intuitive approach at understanding how RNN work, have a look to this interesting blog: by  <a href="https://towardsdatascience.com/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9" target="_blank"> MIchael Phi</a>.Also <a href="https://www.youtube.com/watch?v=LHXXI4-IEns&t=18s" target="_blank">this movie</a> may prove useful.

A very common example used as an introduction to RNN is that of a moving ball. Let us assume that your problem is the prediction of the trajectory of a ball which is moving on a plane. If you have just the current position (epoch $t_1$) of the ball (case A in figure below: NO MEMORY), you cannot say anything about its future movement (epoch $t_2$) since all directions (A, B and C) are qqui-probable. 

<img src="immagini_ANN/balls_1.jpg" alt="drawing" width="40%"/>

But if you have a sequence of data showing its trajectory over time (CaseB in the image), than your prediction can be constrained by using the previous positions of the ball and prediction B becomes way far more probable. This is a very simple example but it has very far reaching implications.

Think now to speech recognition. A spoken text can be considered as a temporal sequence of sounds: if you can follow the evolution of these sounds you can become able to predict the forthcoming sound. 
<img src="immagini_ANN/CNN_speech.png" alt="drawing" width="60%"/>
*the above is a sequence of audio spectrograms which can be fed to a RNN to predict and/or analyse sounds.*

The question now is: how do you implement a sort of *memory* in a network? The trick is in adding a loop that acts as a highway to allow information to flow from one step to the next (left side of mage below).  In other words, it is a single layer network which passes the status to the same network at the subsewuent step and therefore can be visualised as a multilayer feedforward network where each layer corresponds to a different time step. On the right side you have the **unrolled** representation of the same network. 
<img src="immagini_ANN/RNN-unrolled1.png" alt="drawing" width="30%"/>

The hidden state is a representation of previous inputs. What is the use? Let us make an example: a chatbot which needs to react to a sentence typed by an user: *What time is it?*
<img src="immagini_ANN/RNNa_3.gif" alt="drawing" width="30%"/>

First, you initialize your network layers and the initial hidden state. The shape and dimension of the hidden state will be dependent on the shape and dimension of your recurrent neural network. Then you loop through your inputs, pass the word and hidden state into the RNN. The RNN returns the output and a modified hidden state. 
You continue to loop until you’re out of words. Last you pass the output to the feedforward layer, and it returns a prediction. And that’s it! The control flow of doing a forward pass of a recurrent neural network is a for loop.

Unfortunately as you can easily see, the weight of an input decreases with the distance from the current input. This problem is called *short memory* and results from the vanishing gradient problem. 

As the RNN processes more steps, it has troubles retaining information from previous steps. As a matter of fact the fact that we can think of each time step in a recurrent neural network as a layer allows us to train it using a variantr of  back-propagation called **back-propagation through time**. 
The gradient values will exponentially shrink as it propagates through each time step. Again, the gradient is used to make adjustments in the neural networks weights thus allowing it to learn. Small gradients mean small adjustments. That causes the early layers not to learn.

In other words, RNN work well when the problem can be solved by looking only at recent information. For instance, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in *the clouds are in the ...”* we don’t need any further context – it’s obvious that the next word is going to be *sky*. 
The situation would be much more critical if you try to finish a sequence like: *... As a kid I spent many years in Germany, in a small village near the capital where I also attended school and became fluent in xxxxx ...* The missing word is *German* as it could be easily extrapolated from the word *Germany* which, however, is too far away in the sequence. 

RNN, therefore, have two main problems: they are slow to train and, they fail to memorize the information from many time stamps before. The problem, as often happens is in the way backpropagation works.

* Vanishing gradients. In a long sequence, the gradients get multiplied by the weight matrix (transpose) at every time step. If there are small values in the weight matrix, the norm of gradients get smaller and smaller exponentially.Because of vanishing gradients, the RNN doesn’t learn the long-range dependencies across time steps. That means that there is a possibility that the word “what” and “time” are not considered when trying to predict the user’s intention. The network then has to make the best guess with “is it?”. That’s pretty ambiguous and would be difficult even for a human. So not being able to learn on earlier time steps causes the network to have a short-term memory.

* Exploding gradients: If we have a large weight matrix and the non-linearity in the recurrent layer is not saturating, the gradients will explode. The weights will diverge at the update step. We may have to use a tiny learning rate for the gradient descent to work.

### 1.5.2 RNN: a more formal understanding 

Lecun's formalism and accordingly to the following scheme, we have (this is the so called *rolled* representation): 

<img src="immagini_ANN/RNN_rolled_2.png" alt="drawing" width="50%"/>


where:

* $x(t)$ : input that varies with time (a time series)
* ${Enc}(x(t))$: encoder that generates a representation of input
* $h(t)$: a representation of the input
* $w$: trainable parameters
* $z(t-1)$: previous hidden state, which is the output of the previous time step
* $z(t)$: current hidden state
* $g$: function that can be a complicated neural network; one of the inputs is $z(t-1)$: which is the output of the previous time step. $w$ obviously are the weights.
* $Dec(z(t))$: decoder that generates an output

Again, in order to better visualize how these networks can be implemented, it is better to unroll the network. 

<img src="immagini_ANN/RNN_unrolled_4.png" alt="drawing" width="50%"/>

Data are in the form of a sequence: $\mathbf{X}=\{x(0), x(1), x(2), ..., x(n)\}$
The network works as it follows: 

* At time $t=0$, the input $x(0)$ is passed to the encoder and it generates the representation $h(x(0)) = \text{Enc}(x(0))$ and then passes it to $G$ to generate the hidden state $z(0) = G(h_0, z’, w)$. Where $z'$ stands for the inizialization of the model (cf. random). 
* $z(0)$ is passed to the decoder to generate an output,  and also to the next time step.
* the operation is iterated as long as needed (until the sequence stops).

As there are no loops in this network, we can implement backpropagation. But this is a very simple, regular network with one particular characteristic: every block shares the same weights.

RNNs can be used in multiple types of models.

1. **Vector-Sequence Models** — Take fixed-sized vectors as input and output vectors of any size. For example, in image captioning, the image is the input and the output describes the image.

2. **Sequence-Vector Model** — Take a vector of any size and output a vector of fixed size.  For example, sentiment analysis of a movie rates the review of any movie, positive or negative, as a fixed size vector.

3. **Sequence-to-Sequence Model** — The most popular and most used variant, this takes a sequence as input and outputs another sequence with variant sizes. An example of this is language translation for time series data for stock market prediction.

#### 1.5.2.1 Problems with RNNs:

The main reason to use RNNs is for the advantage of remembering information in the past. However, they are slow and fail to memorize the information from many time stamps before. The problem, as it often happens, is in backpropagation which runs into the usual problems:

* **Vanishing gradients:**. In a long sequence, the gradients get multiplied by the weight matrix (transpose) at every time step. If there are small values in the weight matrix, the norm of gradients get smaller and smaller exponentially.Because of vanishing gradients, the RNN doesn’t learn the long-range dependencies across time steps. That means that there is a possibility that the word “what” and “time” are not considered when trying to predict the user’s intention. The network then has to make the best guess with “is it?”. That’s pretty ambiguous and would be difficult even for a human. So not being able to learn on earlier time steps causes the network to have a short-term memory.

* **Exploding gradients**: If we have a large weight matrix and the non-linearity in the recurrent layer is not saturating, the gradients will explode. The weights will diverge at the update step. We may have to use a tiny learning rate for the gradient descent to work.

This problems can sometimes be solved by adopting: 

* **clipping gradients:** (avoid exploding gradients) Squash the gradients when they get too large.

* **Initialization:** (start in right ballpark avoids exploding/vanishing) Initialize the weight matrices to preserve the norm to some extent. For example, orthogonal initialization initializes the weight matrix as a random orthogonal matrix.

**Typical example of the problem**

The input is the characters from a C Program. The system will tell whether it is a syntactically correct program. A syntactically correct program should have a valid number of braces and parentheses. Thus, the network should remember how many open parentheses and braces there are to check, and whether we have closed them all. The network has to store such information in hidden states like a counter. However, because of vanishing gradients, it will fail to preserve such information in a long program.

#### Multiplicative Modules

In multiplicative modules rather than only computing a weighted sum of inputs, we compute products of inputs and then compute weighted sum of that.

Suppose $x \in \mathcal{R}^{n\times1}$, $W \in {R}^{m \times n}$, $U \in {R}^{m \times n \times d}$ and $z \in {R}^{d\times1}$. Here U is a tensor.

$$w_{ij} = u_{ij}^\top z = \begin{pmatrix} u_{ij1} & u_{ij2} & \cdots &u_{ijd}\\ \end{pmatrix} \begin{pmatrix} z_1\\ z_2\\ \vdots\\ z_d\\ \end{pmatrix} = \sum_ku_{ijk}z_k$$

$$s = \begin{pmatrix} s_1\\ s_2\\ \vdots\\ s_m\\ \end{pmatrix} = Wx = \begin{pmatrix} w_{11} & w_{12} & \cdots &w_{1n}\\ w_{21} & w_{22} & \cdots &w_{2n}\\ \vdots\\ w_{m1} & w_{m2} & \cdots &w_{mn} \end{pmatrix} \begin{pmatrix} x_1\\ x_2\\ \vdots\\ x_n\\ \end{pmatrix}$$
 
where $s_i = w_{i}^\top x = \sum_j w_{ij}x_j$.
The output of the system is a classic weighted sum of inputs and weights. Weights themselves are also weighted sums of weights and inputs.



Hypernetwork architecture: weights are computed by another network.

Attention
$x_1$ and $x_2$ are vectors, $w_1$ and $w_2$ are scalars after softmax where $w_1 + w_2 = 1$, and $w_1$ and $w_2$ are between 0 and 1.

$w_1x_1 + w_2x_2$ is a weighted sum of $x_1$ and $x_2$, weighted by coefficients $w_1$ and $w_2$.
By changing the relative size of $w_1$ and $w_2$ we can switch the output of $w_1x_1 + w_2x_2$ to $x_1$ or $x_2$ or some linear combinations of $x_1$ and $x_2$

The inputs can have multiple $x$ vectors (more than $x_1$ and $x_2$). The system will choose an appropriate combination, the choice of which is determined by another variable $z$. An attention mechanism allows the neural network to focus its attention on particular input(s) and ignore the others.

Attention is increasingly important in NLP systems that use transformer architectures or other types of attention.

The weights are data independent because $z$ is data independent.

### 1.5.3 LSTM 

To mitigate this short-term memory problem, two specialized recurrent neural networks were created. One called **Long Short-Term Memory** or **LSTM**s for short. The other is **Gated Recurrent Units** or **GRU**s. LSTMs and GRUs essentially function just like RNN’s, but they are capable of learning long-term dependencies using mechanisms called**gates**. 

These **gates** are different tensor operations that can learn what information to add or remove to the hidden state. Because of this ability, short-term memory is less of an issue for them. 

Long short-term memory (LSTM) networks were invented by Hochreiter and Schmidhuber in 1997 [ArLSTM.1] and around 2007 started to revolutionize speech recognition, outperforming traditional models in certain speech applications. 
In 2009, a **Connectionist Temporal Classification (CTC)**-trained LSTM network was the first RNN to win pattern recognition contests when it won several competitions in connected handwriting recognition [ArLSTM.2]. 
In 2014, the Chinese company Baidu used CTC-trained RNNs to break the 2S09 Switchboard Hub5'00 speech recognition dataset benchmark without using any traditional speech processing methods.

LSTM also improved large-vocabulary speech recognition and text-to-speech synthesis and was used in Google Android. In 2015, Google's speech recognition reportedly experienced a dramatic performance jump of 49% through CTC-trained LSTM. Since then LSTM have broken records for improved machine translation, Language Modeling and Multilingual Language Processing. LSTM combined with convolutional neural networks (CNNs) are also used for automatic image captioning. 

For an introduction to how a LSTM works <a href="https://www.youtube.com/watch?v=8HyCNIVRbSU" target="_blank"> watch this video by Michael Phi</a>.




LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!

All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer.
<img src="immagini_ANN/LSTM1.png" alt="drawing" width="40%"/>

LSTMs also have this chain like structure, but the repeating module has a different structure. Instead of having a single neural network layer, there are four, interacting in a very special way.

<img src="immagini_ANN/LSTM2.png" alt="drawing" width="60%"/>
<img src="immagini_ANN/LSTM3.png" alt="drawing" width="40%"/>
In the above diagram, each line carries an entire vector, from the output of one node to the inputs of others. The pink circles represent pointwise operations, like vector addition, while the yellow boxes are learned neural network layers. Lines merging denote concatenation, while a line forking denote its content being copied and the copies going to different locations.

The key to LSTMs is the cell state, the horizontal line running through the top of the diagram.

The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged.

<img src="immagini_ANN/LSTM4.png" alt="drawing" width="80%"/>

The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates, which are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation.
<img src="immagini_ANN/LSTM6.png" alt="drawing" width="10%"/>
The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!”

An LSTM has three of these gates, to protect and control the cell state.

**first step: throwing away information from the cell state**

The first step in LSTM is to decide what information must be thrown away from the cell state. This decision is made by a sigmoid layer called the “forget gate layer.” It looks at $h_{t−1}$ and $x_t$, and outputs a number between 0 and 1 for each number in the cell state $C_{t−1}$. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.”

Let’s go back to our example of a language model trying to predict the next word based on all the previous ones. In such a problem, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.

<img src="immagini_ANN/LSTM7.png" alt="drawing" width="50%"/>

**second step: decide which information to store in the cell state**

Decide what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the “input gate layer” decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values, C~t, that could be added to the state. In the next step, we’ll combine these two to create an update to the state.

In the example of our language model, we’d want to add the gender of the new subject to the cell state, to replace the old one we’re forgetting.
<img src="immagini_ANN/LSTM8.png" alt="drawing" width="50%"/>



**Third Step: update the old cell state, $C_{t−1}$, into the new cell state $C_t$** 

The previous steps already decided what to do, we just need to actually do it. We multiply the old state by $f_t$, forgetting the things we decided to forget earlier. Then we add $i_t \times \hat{C}_t$. This is the new candidate values, scaled by how much we decided to update each state value.
In the case of the language model, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.

<img src="immagini_ANN/LSTM9.png" alt="drawing" width="50%"/>

**Fourth step: setting the output**

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. Then, we put the cell state through $\tanh$ (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.

For the language model example, since it just saw a subject, it might want to output information relevant to a verb, in case that’s what is coming next. For example, it might output whether the subject is singular or plural, so that we know what form a verb should be conjugated into if that’s what follows next.

<img src="immagini_ANN/LSTM10.png" alt="drawing" width="50%"/>

#### 1.5.3.1 Variants of LSTM

Not all LSTMs are the same. In fact, it seems like almost every paper involving LSTMs uses a slightly different version. The differences are minor, but it’s worth mentioning some of them.

* One popular LSTM variant, introduced by Gers & Schmidhuber (2000), is adding “peephole connections.” This means that we let the gate layers look at the cell state. The diagram below adds peepholes to all the gates, but many papers will give some peepholes and not others.
<img src="immagini_ANN/LSTM11.png" alt="drawing" width="50%"/>

* Another variation is to use coupled forget and input gates. Instead of separately deciding what to forget and what we should add new information to, we make those decisions together. We only forget when we’re going to input something in its place. We only input new values to the state when we forget something older.
<img src="immagini_ANN/LSTM12.png" alt="drawing" width="50%"/>

* A slightly more dramatic variation on the LSTM is the Gated Recurrent Unit, or GRU, introduced by Cho, et al. (2014)[ArLSTMV.4]. It combines the forget and input gates into a single “update gate.” It also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models, and has been growing increasingly popular.

<img src="immagini_ANN/LSTM13.png" alt="drawing" width="50%"/>

These are only a few of the most notable LSTM variants. There are lots of others, like Depth Gated RNNs by Yao, et al. (2015) [ArLSTMV.1]. There’s also some completely different approach to tackling long-term dependencies, like Clockwork RNNs by Koutnik, et al. (2014) [ArLSTMV.2].

Which of these variants is best? Do the differences matter? Greff, et al. (2015) [ArLSTMV.3] do a nice comparison of popular variants, finding that they’re all about the same. Jozefowicz, et al. (2015) tested more than ten thousand RNN architectures, finding some that worked better than LSTMs on certain tasks.


Written down as a set of equations, LSTMs look pretty intimidating (see below) but walking through them step by step in this essay makes them a bit more approachable.

LSTMs were a big step in what we can accomplish with RNNs, but there is much more in recent developments:
* in **Attention** networks, the idea is to let every step of an RNN pick information to look at from some larger collection of information. For example, if you are using an RNN to create a caption describing an image, it might pick a part of the image to look at for every word it outputs (<a href="https://arxiv.org/pdf/1502.03044v2.pdf" target="_blank"> Xu, et al. (2015)</a>) .

* **GRID STMs** by <a href="https://arxiv.org/pdf/1507.01526v1.pdf" target="_blank">Kalchbrenner, et al. (2015</a> seem extremely promising. 
* In **generative models** – such as <a href="https://arxiv.org/pdf/1502.04623.pdf" target="_blank"> Gregor, et al. (2015)</a>, <a href="https://arxiv.org/pdf/1506.02216v3.pdf" target="_blank">Chung, et al. (2015)</a>, or <a href="https://arxiv.org/pdf/1411.7610v3.pdf" target="_blank"> Bayer & Osendorfer (2015)</a> – also seems very interesting. 

### 1.5.4 Going deeper into RNN

This section is taken from : <a href="https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks" target="_blank">this link</a>


**One to One**

<img src="immagini_ANN/RNN_1.png" alt="drawing" width="50%"/>

**One to many**
<img src="immagini_ANN/RNN_2.png" alt="drawing" width="50%"/>

* $f(t)= \sigma_g (W_{f}x_t + U_f h_{t-1)}+b_f$
* $i_t = \sigma_g (W_ix_t +U_ih_{t-1}+b_i$
* $o_t = \sigma_o (W_o x_t +U_oh_{t-1}+b_o)$
* $c_t = f_t \odot c_{t-1}+i_t \odot \tanh(W_c x_t + U_c h_{t-1}+b_c)$
* $h_t= o_t \odot \tanh(c_t)$

where $\odot$ is the hadamard (element wise) product, $x_t \in \mathbb{R}^a$  is an input vector to the LSTM unit, $f_t \in \mathbb{R}^h$ is the forget gate's activation vector, $i_t \in \mathbb{R}^h$ is the input/update gate’s activation vector, $o_t \in \mathbb{R}^h$ is the output gate’s activation vector, $h_t\in\mathbb{R}^h$ is the hidden state vector (also known as output), $c_t\in\mathbb{R}^h$ is the cell state vector. 

An LSTM unit uses a cell state $c_t$ to convey the information through the unit. It regulates how information is preserved or removed from the cell state through structures called gates. The forget gate $f_t$ decides how much information we want to keep from the previous cell state $c_{t-1}$
by looking at the current input and previous hidden state, and produces a number between 0 and 1 as the coefficient of $c_{t-1}$.

$\tanh(W_cx_t + U_ch_{t-1} + b_c)$ computes a new candidate to update the cell state, and like the forget gate, the input gate $i_t$ decides how much of the update to be applied. Finally, the output $h_t$ will be based on the cell state $c_t$, but will be put through a $\tanh$ then filtered by the output gate $o_t$.

Though LSTMs are widely used in NLP, their popularity is decreasing. For example, speech recognition is moving towards using temporal CNN, and NLP is moving towards using transformers.

## References

* ArRNN.1 <a href="https://www.pnas.org/doi/pdf/10.1073/pnas.79.8.2554" target="_blank"> Neural networks and physical systems with emergent collective computational abilities </a> J.J. Hopfield, 1982, Proc. NatL Acad. Sci. USA, Vol. 79, pp. 2554-2558, Biophysics
* ArRNN.2 <a href="https://www.nature.com/articles/323533a0.pdf?origin=ppub" target="_blank"> Learning representations by back-propagating errors</a> Williams, Ronald J.; Hinton, Geoffrey E.; Rumelhart, David E., Nature. 323 (6088): 533–536. 1986
* ArLSTM.1 <a href="http://www.bioinf.jku.at/publications/older/2604.pdf" target="_blank">Long Short Term Memory</a>Neural Computation 9(8):1735-1780, Hochreiter and Schmidhuber (1997)
* ArLSTM.2 <a href="https://www.cs.toronto.edu/~graves/nips_2008.pdf" target="_blank">Offline Handwriting Recognition with Multidimensional Recurrent Neural Networks</a>. Graves, Alex; Schmidhuber, Jürgen . in  Advances in Neural Information Processing Systems. Neural Information Processing Systems (NIPS) Foundation. Koller, D.; Schuurmans, D.; Bengio, Y.; Bottou, L. (eds.), 21: 545–55 (2009)
* ArLSTMV.1 <a href="https://arxiv.org/pdf/1508.03790.pdf" target="_blank">Depth-Gated LSTM </a>arXiv:1508.03790v4 (2015)
* ArLSTMV.2 <a href="https://arxiv.org/pdf/1402.3511.pdf" target="_blank"> A clockwork RNN</a> J. Koutnik et al. (2014)
* ArLSTMV.3 <a href="https://arxiv.org/pdf/1503.04069.pdf" target="_blank">LSTM: A Search Space Odyssey</a>Greff, et al. (2015). A comparison of many LSTM models.